# 03 - Fake vs Real News Classification - Embedding-based Model

This notebook explores an alternative approach to the TF-IDF model by using word embeddings to represent the text data.

## Project Goal
Build a classifier to distinguish between real (1) and fake (0) news articles using word embeddings.

## Workflow
1. Load preprocessed training and testing data.
2. Use a pre-trained language model from `spaCy` to generate document embeddings.
3. Train multiple classification models on the embeddings.
4. Evaluate and compare model performance against the TF-IDF models.
5. Select the best performing model and save it.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import spacy
import joblib
#from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB # Using GaussianNB for continuous features
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, classification_report)
import time
import os

# Load the spaCy model
nlp = spacy.load('en_core_web_md')

print("Libraries and spaCy model loaded successfully!")

Libraries and spaCy model loaded successfully!


In [4]:
# Load the preprocessed training and testing data
print("Loading preprocessed data...")

train_data = pd.read_csv('../data/processed/train.csv')
test_data = pd.read_csv('../data/processed/test.csv')


Loading preprocessed data...


In [5]:
train_data.head()

,text_processed,label,title_length,text_length,title_word_count,text_word_count
0,hear presid trump doubl condemn evil kkk white...,0,125,3940,18,677
1,wow christian author give unexpect brilliant a...,0,157,1874,25,324
2,obama black live matter terrorist join peopl l...,0,160,7174,24,1177
3,brexit deal risk chao drug suppli report warn ...,1,60,1434,11,224
4,insan iowa republican liter want let toddler c...,0,66,2836,10,496


In [7]:

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")


X_train_text = train_data['text_processed']
y_train = train_data['label']
X_test_text = test_data['text_processed']
y_test = test_data['label']

print(f"Data loaded")

Training data shape: (31948, 6)
Testing data shape: (7987, 6)
Data loaded


In [8]:
# Generate document embeddings
print("Generating document embeddings...")

def get_embedding(text):
    """Generate a document embedding by averaging word vectors."""
    doc = nlp(text)
    # We filter out stop words and punctuation, and only use tokens with vectors.
    vectors = [token.vector for token in doc if token.has_vector and not token.is_stop and not token.is_punct]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        # If no vectors are found, return a zero vector of the same dimension.
        return np.zeros(nlp.meta['vectors']['width'])

# Apply the function to the text columns
# This may take a few minutes to run
X_train = np.array([get_embedding(text) for text in X_train_text])
X_test = np.array([get_embedding(text) for text in X_test_text])

print(f"Embeddings generated successfully!")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

Generating document embeddings...
Embeddings generated successfully!
X_train shape: (31948, 300)
X_test shape: (7987, 300)


In [ ]:
# create and save tables with embeddings and labels
train_embeddings_df = pd.DataFrame(X_train)
train_embeddings_df['label'] = y_train.values
train_embeddings_df.to_csv('../data/processed/train_embeddings_md_spacy.csv', index=False)
test_embeddings_df = pd.DataFrame(X_test)
test_embeddings_df['label'] = y_test.values
test_embeddings_df.to_csv('../data/processed/test_embeddings_md_spacy.csv', index=False)
print("Embeddings saved to CSV files.")

Embeddings saved to CSV files.


In [13]:
train_embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
0,-0.710770,0.077117,-0.053336,-0.085966,-0.105871,-0.067968,0.042675,-0.011412,0.023816,1.532586,...,0.076064,-0.127728,-0.105520,0.109161,0.038407,-0.044421,-0.028024,-0.045777,0.097853,0
1,-0.721213,0.050481,-0.063265,-0.052389,-0.064379,-0.018971,-0.001621,0.011643,0.078975,1.374136,...,-0.003770,-0.240199,-0.106126,0.163552,0.003189,-0.033027,-0.014192,-0.024645,0.077795,0
2,-0.720840,0.085728,0.014629,-0.076570,-0.092617,-0.025558,0.012029,0.069394,-0.018650,1.326825,...,0.032615,-0.081762,-0.100539,0.084443,0.123469,-0.012912,-0.014758,-0.052218,0.085369,0
3,-0.732216,0.121917,-0.037588,-0.104549,-0.128978,-0.091884,0.070541,-0.054250,-0.047675,1.502787,...,0.080932,-0.048728,-0.028285,0.130951,0.039130,-0.048612,-0.078373,0.017870,0.062668,1
4,-0.739269,0.079492,-0.036488,-0.072660,-0.134054,-0.084150,0.067199,-0.041604,0.026031,1.251597,...,-0.025673,-0.141390,0.014278,0.127763,0.059271,-0.066245,-0.003457,-0.017368,0.054992,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31943,-0.738895,0.057703,0.055241,0.065791,-0.187318,-0.002069,-0.004653,-0.005006,0.019130,1.210082,...,0.063323,-0.085839,-0.084856,0.108549,0.036419,-0.092900,-0.029374,-0.072611,0.080534,0
31944,-0.716590,0.040969,0.013343,0.007109,-0.121197,-0.078309,0.043503,-0.015462,-0.044129,0.986988,...,-0.008687,-0.141480,-0.112583,0.123442,-0.000089,0.003843,-0.023963,0.005707,0.000379,1
31945,-0.774491,0.079623,0.048903,-0.045217,-0.202914,-0.038544,0.011340,-0.012785,0.053201,1.581481,...,0.052390,-0.050179,-0.101415,0.049589,0.098197,-0.142524,-0.093298,0.104006,-0.029149,1
31946,-0.734738,0.139330,-0.031314,-0.095167,-0.133841,-0.024863,0.106466,-0.024880,0.054591,1.583399,...,0.063654,0.022584,-0.032288,0.138828,-0.014331,-0.001135,-0.024683,0.046092,-0.031402,0


In [9]:
# Define machine learning models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'Gaussian Naive Bayes': GaussianNB()
}

print(f"Defined {len(models)} models for training.")

Defined 4 models for training.


In [10]:
# Train and evaluate each model
print("Training and evaluating models...")
print("=" * 60)

results = {}

for name, model in models.items():
    print(f"\n🔄 Training {name}...")
    start_time = time.time()
    
    # Fit the model on training data
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Make predictions on test set
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')
    
    # Store results
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'training_time': training_time
    }
    
    # Display results
    print(f"  ✓ Training completed in {training_time:.1f} seconds")
    print(f"    Accuracy:    {accuracy:.4f}")
    print(f"    Precision:   {precision:.4f}")
    print(f"    Recall:      {recall:.4f}")
    print(f"    F1-Score:    {f1:.4f}")

print(f"\n🎯 Model training completed!")

Training and evaluating models...

🔄 Training Logistic Regression...
  ✓ Training completed in 0.7 seconds
    Accuracy:    0.9253
    Precision:   0.9165
    Recall:      0.9360
    F1-Score:    0.9262

🔄 Training Random Forest...
  ✓ Training completed in 41.9 seconds
    Accuracy:    0.9135
    Precision:   0.9113
    Recall:      0.9165
    F1-Score:    0.9139

🔄 Training SVM...
  ✓ Training completed in 422.9 seconds
    Accuracy:    0.9345
    Precision:   0.9241
    Recall:      0.9470
    F1-Score:    0.9354

🔄 Training Gaussian Naive Bayes...
  ✓ Training completed in 0.1 seconds
    Accuracy:    0.8217
    Precision:   0.8038
    Recall:      0.8520
    F1-Score:    0.8272

🎯 Model training completed!


In [11]:
# Create model comparison table
comparison_df = pd.DataFrame(results).T.drop(columns=['model'])
print("📊 MODEL COMPARISON (Embeddings)")
print("=" * 80)
print(comparison_df.round(4))

# Identify best model based on F1-score
best_model_name = comparison_df['f1'].idxmax()
best_model = results[best_model_name]['model']
print(f"\n🏆 Best performing model: {best_model_name}")

# Save the best model
os.makedirs('../outputs/model', exist_ok=True)
model_filename = f'../outputs/model/best_embedding_model_{best_model_name.lower().replace(" ", "_")}.pkl'
joblib.dump(best_model, model_filename)
print(f"✓ Best model saved to: {model_filename}")

📊 MODEL COMPARISON (Embeddings)
                      accuracy precision  recall        f1 training_time
Logistic Regression   0.925254  0.916524   0.936   0.92616      0.729384
Random Forest         0.913484   0.91126  0.9165  0.913873     41.906513
SVM                   0.934519  0.924128   0.947  0.935424    422.932141
Gaussian Naive Bayes   0.82171  0.803774   0.852  0.827184      0.096117

🏆 Best performing model: SVM
✓ Best model saved to: ../outputs/model/best_embedding_model_svm.pkl


## Results and Comparison

The table above shows the performance of different classifiers trained on the document embeddings. 

### Comparison with TF-IDF Models

Let's compare these results with the TF-IDF based models from the previous notebook.

| Model (TF-IDF)         | Accuracy | F1-Score |
|------------------------|----------|----------|
| Logistic Regression    | 0.9850   | 0.9850   |
| Random Forest          | 0.9932   | 0.9933   |
| Naive Bayes            | 0.9395   | 0.9395   |
| Support Vector Machine | 0.9922   | 0.9923   |

**Observations:**

* The TF-IDF models, especially Random Forest and SVM, achieved near-perfect scores, which is suspicious given the nature of the dataset.
* The embedding-based models have lower scores, which might be a more realistic representation of the model's performance on unseen data.
* The difference in performance suggests that the TF-IDF approach might be overfitting to the specific vocabulary of the training data.
* The embedding-based approach, by capturing semantic meaning, might be more robust and generalize better, even with a lower score on this specific test set.

**Conclusion:**

While the TF-IDF models show higher metrics, the embedding-based models are likely to be more reliable in a real-world scenario. The choice between the two would depend on the specific goals of the project. If the goal is to have a model that understands the content better and is less susceptible to simple keyword manipulation, the embedding-based model is a better choice.